In [14]:
#Instalar el paquete que controla el navegador
!pip install selenium
!pip install Dropbox
#Descargar el Driver en :https://sites.google.com/a/chromium.org/chromedriver/downloads

#importar y chequer si selenium funciona

from selenium import webdriver
from selenium.webdriver.common.keys import Keys as ky
from selenium.webdriver.support.ui import WebDriverWait     
from selenium.webdriver.common.by import By     
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
import openpyxl
from openpyxl import load_workbook
import re
import itertools
import time 
import pandas as pd
import dropbox

# selecionando nuestro webbrowser (selecionar la ubicación donde este el driver)
try:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver80")
except:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver85")
    print("El driver fue instalado correctamente")

El driver fue instalado correctamente


In [15]:
#pagina donde va a buscar
#1. Ktronix
driver.maximize_window()

docs = openpyxl.load_workbook ("Categorias.xlsx")
hoja = docs.get_sheet_by_name("ktronix")

itemsK = []
for columna in hoja.iter_rows():
    bien = columna[0].value
    itemsK.append(bien)

linksK = []
priceK = []
descK = []
codesK = []
timeK = []
obj = []
timeKF = []
driver.get("https://www.ktronix.com/")

for item in itemsK:
    searchBar = WebDriverWait(driver, 200).until(lambda x: x.find_element_by_id("js-site-search-input")) #Selecionar la barra de busqueda
    searchBar.send_keys(item) #escribir en la barra de busqueda
    searchBar.submit() #Enviar la busqueda sortNumber1
    print(item)
    sort = WebDriverWait(driver, 20).until(lambda x: x.find_element_by_id("sortNumber1"))
    sort.send_keys("50")
    #sort = WebDriverWait(driver, 200).until(lambda x: x.find_element_by_id("sortNumber1"))
    #sort.send_keys("50") #amplia la busqueda a 50 items por subcategoria
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    elemsp = WebDriverWait(driver, 200).until(lambda x: x.find_elements_by_class_name("price")) #busca y guarda los Webelemet con los precios en una lista
    #Agregar pricedis
    for elemp in elemsp:
        if elemp.text != "":
            ele = elemp.text #obtiene el precio del webelement
            value = re.findall(r"[0-9.]*[0-9]+", ele) #quita el signo "$" del precio
            priceK.append(value[0]) #guarda el precio en una lista
        else:
            continue
    elems = WebDriverWait(driver, 200).until(lambda x: x.find_elements_by_class_name("js-product-click-datalayer")) #busca y guarda los  Webelements con los links en una lista
    for elem in itertools.islice(elems,None,None,3): #realiza un loop cada "3" Webelements
        linksK.append(elem.get_attribute("href")) #Obtiene y guarda el webelement en la lista "linksK"
        timeK.append(time.strftime("%d/%m/%y %H:%M:%S"))
        timeKF.append(time.strftime("%d/%m/%y"))
        obj.append(item)
    #except:
        #print("Se genero un error")
        #continue
        
for link in linksK: #Obtener descripcion & Codigo unico de los links
    des = re.findall(r"com.(.*?).p/", link)
    des = des[0]
    descK.append(des) #guarda cada descripcion en la lista "descK"
    code = re.findall(".*?([0-9]+)$",link)
    code = code[0]
    codesK.append(code) #guarda cada descripcion en la lista "codesK"

<ipython-input-15-b9f866fa2954>:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  hoja = docs.get_sheet_by_name("ktronix")


Celular
Computador portatil
Computador escritorio
Televisor
Audio
Camara
Consola
Nevera
Lavadora
Estufa
Ventilador
Aspiradora
plancha de ropa
licuadora
sanduchera
Bicicleta
Caminadora
Balon


In [16]:
len(priceK)

845

In [4]:
len(linksK)

870

In [5]:
len(timeK)

870

In [38]:
len(descK)

870

In [39]:
len(codesK)

870

In [40]:
len(obj)

870

In [7]:
nowa = time.strftime("%d-%m-%y")
dataf = pd.DataFrame({"Codigo":codesK, "Descripción":descK,
                      "Subcategoria":obj, "Precio":priceK,
                      "Link":linksK, "Fecha & Hora":timeK, "Fecha":timeKF})
book = load_workbook('Ktronix.xlsx')
writer = pd.ExcelWriter('Ktronix.xlsx', engine = 'openpyxl')
writer.book = book
dataf.to_excel(writer, sheet_name = ("Ktronix " + str(nowa)), index = False)
writer.save()

In [2]:
xls = pd.ExcelFile("C:/Users/Ivan/Documents/CCCE/Ktronix/Ktronix.xlsx")

In [3]:
names1 = xls.sheet_names

In [12]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

my_value = pd.DataFrame()
merge_value = pd.DataFrame()
for i in names1:
    my_value = pd.read_excel(r"C:\Users\Ivan\Documents\CCCE\Ktronix\Ktronix.xlsx", sheet_name = i)
    merge_value = pd.concat([merge_value,my_value], ignore_index = False)
#print(merge_value.head)
merge_value.to_csv("Ktronix Consolidado.csv", index = False)

# Importar a dropbox

In [13]:
import dropbox
from dropbox.files import WriteMode

cliente = dropbox.Dropbox('sl.Ak9Xgcw8jKOKY9oqvWvcLIKykrRLgbZdFJ8A-ERmUjoYSzyz9f90Ae6ZybmJ_f5EhvscXuicOXQpB0ao6E2FiiH85TdKg3tqfIaYTUac12-rMvospyfQ-obONJULytZK2v6l4dxYH_Y')
path = "/Dropbox/Aplicaciones/save_python/Ktronix Consolidado.csv"
#print(cliente.users_get_current_account()) 
f= open("C:/Users/Ivan/Documents/CCCE/Ktronix/Ktronix Consolidado.csv", "rb")
respuesta = cliente.files_upload(f.read(),path, mode = dropbox.files.WriteMode.overwrite)

In [24]:
Today = str(time.strftime("%Y/%m/%d"))
dates = pd.date_range(start= "16/10/20", end = Today)
df = pd.DataFrame()
for date in dates:
    df[date] = []

In [25]:
df.head()

,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04


In [32]:
for i in dates:
    print(i)

2020-10-16 00:00:00
2020-10-17 00:00:00
2020-10-18 00:00:00
2020-10-19 00:00:00
2020-10-20 00:00:00
2020-10-21 00:00:00
2020-10-22 00:00:00
2020-10-23 00:00:00
2020-10-24 00:00:00
2020-10-25 00:00:00
2020-10-26 00:00:00
2020-10-27 00:00:00
2020-10-28 00:00:00
2020-10-29 00:00:00
2020-10-30 00:00:00
2020-10-31 00:00:00
2020-11-01 00:00:00
2020-11-02 00:00:00
2020-11-03 00:00:00
2020-11-04 00:00:00


In [9]:
xls = pd.ExcelFile("C:/Users/Ivan/Documents/CCCE/Ktronix/Ktronix.xlsx")

In [10]:
names1 = xls.sheet_names

In [15]:
fechas = []
for date in names1[1:]:
    value = re.findall(r"[0-9-]*[0-9]+", date)
    print(value)
    value = value[0]
    fechas.append(value)
print(fechas)

['16-10-20']
['18-10-20']
['19-10-20']
['20-10-20']
['21-10-20']
['22-10-20']
['23-10-20']
['26-10-20']
['27-10-20']
['28-10-20']
['29-10-20']
['30-10-20']
['01-11-20']
['02-11-20']
['03-11-20']
['04-11-20']
['16-10-20', '18-10-20', '19-10-20', '20-10-20', '21-10-20', '22-10-20', '23-10-20', '26-10-20', '27-10-20', '28-10-20', '29-10-20', '30-10-20', '01-11-20', '02-11-20', '03-11-20', '04-11-20']


In [3]:
col = [2]
Today = str(time.strftime("%Y-%m-%d"))
dates = pd.date_range(start= "16/10/20", end = Today)
for date in dates:
    
    if date == 
    df = pd.read_excel(r"C:\Users\Ivan\Documents\CCCE\Ktronix\Ktronix.xlsx", sheet_name = "Ktronix 16-10-20", index_col = "Codigo")
df = df.iloc[:, 2:3]

In [4]:
df.head()

,Precio
Codigo,
7707222705548,1.099.900
8806090219580,1.199.900
723755137424,529.900
6941059635529,719.900
7707222707979,1.099.900


In [5]:
df.head

<bound method NDFrame.head of                   Precio
Codigo                  
7707222705548  1.099.900
8806090219580  1.199.900
723755137424     529.900
6941059635529    719.900
7707222707979  1.099.900
...                  ...
34264424166      176.000
842978134024      36.900
50875823143       97.900
768686283352     409.800
768686283680     409.800

[850 rows x 1 columns]>

In [2]:
xls = pd.ExcelFile("C:/Users/Ivan/Documents/CCCE/Ktronix/Ktronix.xlsx")

In [3]:
names1 = xls.sheet_names

In [55]:
print(names1)

['Ktronix 02-10-20', 'Ktronix 16-10-20', 'Ktronix 18-10-20', 'Ktronix 19-10-20', 'Ktronix 20-10-20', 'Ktronix 21-10-20', 'Ktronix 22-10-20', 'Ktronix 23-10-20', 'Ktronix 26-10-20', 'Ktronix 27-10-20', 'Ktronix 28-10-20', 'Ktronix 29-10-20', 'Ktronix 30-10-20', 'Ktronix 01-11-20', 'Ktronix 02-11-20', 'Ktronix 03-11-20']


In [70]:
#merge_value.head
import openpyxl
from openpyxl import load_workbook
book = load_workbook('Ktronix.xlsx')
writer = pd.ExcelWriter('Ktronix.xlsx', engine = 'openpyxl')
writer.book = book
merge_value.to_excel(writer, sheet_name = "Ktronix consolidado", index = False)
writer.save()

In [71]:
merge_value.head()

,Codigo,Descripción,Subcategoria,Precio,Link,Fecha & Hora
0,7707222705548,celular-samsung-galaxy-m31-128gb-negro-cover-n...,Celular,1.099.900,https://www.ktronix.com/celular-samsung-galaxy...,02/10/20 09:36:48
1,8806090219580,celular-samsung-galaxy-a30s-128gb-morado,Celular,1.199.900,https://www.ktronix.com/celular-samsung-galaxy...,02/10/20 09:36:48
2,723755137424,celular-motorola-one-macro-64gb-azul,Celular,529.900,https://www.ktronix.com/celular-motorola-one-m...,02/10/20 09:36:48
3,6941059635529,celular-xiaomi-redmi-note-8-128gb-azul,Celular,719.900,https://www.ktronix.com/celular-xiaomi-redmi-n...,02/10/20 09:36:48
4,7707222707979,celular-samsung-galaxy-m31-128gb-azul-cover-negro,Celular,1.099.900,https://www.ktronix.com/celular-samsung-galaxy...,02/10/20 09:36:48
